<a href="https://colab.research.google.com/github/ryu622/gnn-counterattack-xai-v2/blob/fix%2Ffile-clean/GAT_CounterAttack_Prediction_Train_Scientific8_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#シード値
import random
import os
import numpy as np
import torch

def set_seed(seed=42):
    # Python自体の乱数固定
    random.seed(seed)
    # OS環境の乱数固定
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Numpyの乱数固定
    np.random.seed(seed)
    # PyTorchの乱数固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # マルチGPUの場合
    # 計算の決定論的挙動を強制（これを入れると少し遅くなることがありますが、再現性は完璧になります）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 好きな数字（42が一般的）で固定
set_seed(42)

In [ ]:
#GoogleDriveをマウント

from google.colab import drive

# Google Driveを仮想ファイルシステムにマウント
drive.mount('/content/drive')

In [ ]:
# 必須モジュールのインポート
!pip install torch_geometric

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import optim

import pandas as pd
import os
from glob import glob
import numpy as np
import re
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# 表示設定
np.set_printoptions(suppress=True, precision=3)
pd.set_option('display.precision', 3)    # 小数点以下の表示桁
pd.set_option('display.max_rows', 50)   # 表示する行数の上限
pd.set_option('display.max_columns', 15)  # 表示する列数の上限
%precision 3

使用データは以前までと同様

In [ ]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report
from collections import defaultdict
import random

# ==========================================
# 1. 補助関数: 試合単位のアンダーサンプリング
# ==========================================
def balance_dataset_by_undersampling(data_list):
    """
    シークエンス単位で1:1に調整。訓練データにのみ適用。
    """
    seq_groups = defaultdict(list)
    for d in data_list:
        sid = int(d.sequence_id.item()) if torch.is_tensor(d.sequence_id) else int(d.sequence_id)
        seq_groups[sid].append(d)

    l0_groups, l1_groups = [], []
    for sid, frames in seq_groups.items():
        label = int(frames[0].y.item())
        if label == 0: l0_groups.append(frames)
        else: l1_groups.append(frames)

    # 少ない方（成功）に合わせて失敗を削る
    min_size = min(len(l0_groups), len(l1_groups))
    sampled_l0 = random.sample(l0_groups, min_size)
    sampled_l1 = l1_groups # 成功は全数保持

    balanced_list = [frame for group in (sampled_l0 + sampled_l1) for frame in group]
    random.shuffle(balanced_list)

    print(f"    [Sampling] Success Seqs: {len(sampled_l1)} | Failure Seqs: {len(sampled_l0)} | Total Frames: {len(balanced_list)}")
    return balanced_list



モデルの定義（PIGNNのオリジナルクラス）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
from torch_geometric.data import Data
import os

# ==========================================
# 1. 前処理関数の定義
# ==========================================
def preprocess_batch(data, device):
    # スケーリングはBuilder側で行われているため、ここでは型変換とDevice転送に集中
    data.x = data.x.float()
    data.pos = data.pos.float()
    data.vel = data.vel.float()
    return data.to(device)

# ==========================================
# 2. モデル定義（チーム属性によるメッセージ分岐の実装）
# ==========================================
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index, pos, vel):
        h = self.lin(x)
        # チームID(index 6)をメッセージパッシングに渡す
        return self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 6:7])

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        # 理論1: 未来位置予測ベースのバイアス
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        # 【追加】チーム関係分岐: 味方なら+0.5, 敵なら-0.5の注目度補正
        is_teammate = (team_i == team_j).float()
        team_bias = torch.where(is_teammate > 0.5, 0.5, -0.5)

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        # 物理バイアス + チーム属性バイアス を統合
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = self.conv2(x, edge_index, pos, vel)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

# ==========================================
# 3. 理論修正：集団運動学的制約（L_phys）
# ==========================================
def pignn_theoretical_loss(output, target, data, alpha=0.1):
    # L_task: クラス重み付き（Success:3.3倍）
    weights = torch.tensor([1.0, 3.3], device=output.device)
    loss_task = F.nll_loss(output, target, weight=weights)

    # L_phys: 集団推進力制約
    # Success確率が高いほど、チーム全体の重心(全ノード平均)が右(+vx)であることを求める
    probs = torch.exp(output)[:, 1]

    # グラフごとの平均vxを算出（選手全員の動きを統合）
    # data.batch を用いて各グラフ(シーン)の平均vxを計算
    batch_size = output.size(0)
    # 各グラフの平均vxを計算
    avg_vxs = []
    for i in range(batch_size):
        mask = (data.batch == i)
        avg_vxs.append(torch.mean(data.vel[mask, 0])) # 各シーンの全ノード平均vx

    avg_vxs = torch.stack(avg_vxs)

    # 物理損失：Success確率 × ReLU(-平均vx)
    # シーン全体が左に流れているのに「成功」と出すと強く罰せられる
    loss_phys = torch.mean(probs * torch.relu(-avg_vxs))

    total_loss = loss_task + (alpha * loss_phys)
    return total_loss, loss_task, loss_phys

# ==========================================
# 4. 学習・評価ループ
# ==========================================
'''
def train_pignn_epoch_dynamic(model, loader, optimizer, device, epoch):
    model.train()
    total_loss, total_phys = 0, 0

    # アニーリング（後半10エポック以降で物理を強化）
    if epoch <= 10:
        current_alpha = 0.1
    else:
        current_alpha = min(0.1 + (epoch - 10) * 0.2, 3.0)

    for data in loader:
        data = preprocess_batch(data, device)
        optimizer.zero_grad()

        out = model(data)
        loss, _, l_phys = pignn_theoretical_loss(out, data.y.view(-1), data, alpha=current_alpha)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        total_phys += l_phys.item() * data.num_graphs

    return total_loss / len(loader.dataset), total_phys / len(loader.dataset), current_alpha'''

def train_pignn_epoch_fixed(model, loader, optimizer, device, alpha_p):
    """
    論文の実験(A案)を再現するための、alpha固定学習ループ。
    """
    model.train()
    total_loss, total_phys = 0, 0

    # 動的な変更を廃止し、引数で受け取った固定値を使用
    current_alpha = alpha_p

    for data in loader:
        data = preprocess_batch(data, device)
        optimizer.zero_grad()

        out = model(data)
        # 修正されたalphaを損失関数に渡す
        loss, _, l_phys = pignn_theoretical_loss(out, data.y.view(-1), data, alpha=current_alpha)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        total_phys += l_phys.item() * data.num_graphs

    return total_loss / len(loader.dataset), total_phys / len(loader.dataset), current_alpha

def test_pignn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 幽霊データ排除版
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 1.0

v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v16_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
# 既に保存側で全データに match_id が付与されている前提です
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV = 30
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f" Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    # ※balance_dataset_by_undersampling 関数は以前のものをそのまま使用
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        # アンパックエラーを避けるため戻り値3つを正しく受け取る
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    # 保存ルートディレクトリ
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    # αごとの専用サブフォルダを作成
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}")
    os.makedirs(alpha_dir, exist_ok=True)

    # 1. 各試合(Round)ごとの重みを保存
    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    # 2. そのαにおける「最高傑作」を保存
    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f" [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        torch.save(cv_model.state_dict(), f'best_pignn_alpha_{FIXED_ALPHA}.pth')

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:    {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 幽霊データ排除版
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 0

v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v16_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
# 既に保存側で全データに match_id が付与されている前提です
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV = 30
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f" Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    # ※balance_dataset_by_undersampling 関数は以前のものをそのまま使用
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        # アンパックエラーを避けるため戻り値3つを正しく受け取る
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    # 保存ルートディレクトリ
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    # αごとの専用サブフォルダを作成
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}")
    os.makedirs(alpha_dir, exist_ok=True)

    # 1. 各試合(Round)ごとの重みを保存
    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    # 2. そのαにおける「最高傑作」を保存
    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f" [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        torch.save(cv_model.state_dict(), f'best_pignn_alpha_{FIXED_ALPHA}.pth')

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:    {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f"Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f"Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f" PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f" 物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 1.0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f"Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f" PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f" 物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import torch
from torch_geometric.data import Data

def run_leakage_diagnostic(model, loader, device):
    model.eval()
    # 特徴量ラベル: 0:x, 1:y, 2:vx, 3:vy, 4:dist_goal, 5:dist_ball, 6:team_id
    feature_names = ["x", "y", "vx", "vy", "dist_goal", "dist_ball", "team_id"]

    print(f"{'Removed Feature':<15} | {'Test Acc':<10} | {'Recall (S)':<10}")
    print("-" * 45)

    # ベースライン（全特徴量あり）
    base_acc = test_pignn(model, loader, device)
    print(f"{'None (Baseline)':<15} | {base_acc:.4f}")

    with torch.no_grad():
        for i in range(7):
            correct = 0
            tp = 0 # True Positive
            fn = 0 # False Negative

            for data in loader:
                data = preprocess_batch(data, device)

                # 特定の特徴量をゼロに置き換える
                x_shuffled = data.x.clone()
                x_shuffled[:, i] = 0.0

                # 推論
                out = model(Data(x=x_shuffled, edge_index=data.edge_index,
                                 batch=data.batch, pos=data.pos, vel=data.vel))
                pred = out.argmax(dim=1)

                # 精度計算
                y_true = data.y.view(-1)
                correct += (pred == y_true).sum().item()

                # Recall (Success) 計算
                tp += ((pred == 1) & (y_true == 1)).sum().item()
                fn += ((pred == 0) & (y_true == 1)).sum().item()

            acc = correct / len(loader.dataset)
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            print(f"{feature_names[i]:<15} | {acc:.4f}     | {recall:.4f}")

# 実行
run_leakage_diagnostic(model, test_loader, device)

ただのMLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report
import os
import numpy as np

# ==========================================
# 1. MLPモデルの定義 (PIGNNと入力を完全に揃える)
# ==========================================
class SimpleMLPClassifier(nn.Module):
    def __init__(self, in_channels=7, hidden_channels=64): # あなたのデータ(7次元)に合わせる
        super(SimpleMLPClassifier, self).__init__()
        # グラフ畳み込みを行わず、全結合層のみで判定
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.BatchNorm1d(hidden_channels), # 勾配消失を防ぎ学習を安定化
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 2)
        )

    def forward(self, data):
        # 重要なポイント:
        # メッセージパッシング（近接選手の相互作用理解）をバイパスし、
        # 全選手の平均的な統計量だけで予測を行う
        x = global_mean_pool(data.x, data.batch) # グラフ構造を無視した平均化
        return F.log_softmax(self.mlp(x), dim=1)

# ==========================================
# 2. MLP用のクロスバリデーション実行関数
# ==========================================
def run_mlp_cv(all_data_list, match_ids, device):
    FIXED_ALPHA = "MLP"
    # 保存先を独立させる
    model_save_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline"
    os.makedirs(model_save_dir, exist_ok=True)

    cv_final_reports = []
    print(f"MLP Baseline CV開始 (Input: 7 channels)\n")

    for test_match in match_ids:
        print(f"Round: Match {test_match} (MLP)")

        # PIGNNと全く同じルールで切り分け
        test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
        train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

        # 論文[cite: 105]同様、訓練データのみ1:1にアンダーサンプリング
        cv_train_set = balance_dataset_by_undersampling(train_candidates)

        cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
        cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

        # モデル初期化 (7次元)
        model = SimpleMLPClassifier(in_channels=7).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

        # 30エポック学習 (物理損失なしの標準的な学習)
        model.train()
        for epoch in range(1, 31):
            for d in cv_train_loader:
                d = d.to(device)
                optimizer.zero_grad()
                out = model(d)
                # 論文同様[cite: 115]の50/50ベースラインに近い条件でNLLLossを使用
                loss = F.nll_loss(out, d.y.view(-1))
                loss.backward()
                optimizer.step()

        # 評価 (実戦の不均衡比率のままテスト)
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for d in cv_test_loader:
                d = d.to(device)
                out = model(d)
                y_true.extend(d.y.view(-1).cpu().numpy())
                y_pred.extend(out.argmax(dim=1).cpu().numpy())

        # レポート生成
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
        cv_final_reports.append({
            'match': test_match,
            'recall': report['1']['recall'],
            'precision': report['1']['precision'],
            'f1': report['1']['f1-score']
        })

        # 保存 (後で可視化比較に使う)
        torch.save(model.state_dict(), os.path.join(model_save_dir, f'mlp_match_{test_match}.pth'))
        print(f" >> Result: Precision={report['1']['precision']:.4f}, F1={report['1']['f1-score']:.4f}")

    # 全体の平均を計算
    avg_precision = np.mean([r['precision'] for r in cv_final_reports])
    avg_f1 = np.mean([r['f1'] for r in cv_final_reports])

    print(f"\nMLP CV Final Results")
    print(f"Avg Precision: {avg_precision:.4f}")
    print(f"Avg F1-score:  {avg_f1:.4f}")

    return cv_final_reports

# 実行
mlp_results = run_mlp_cv(all_data_list, match_ids, device)

In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. 設定：MLPモデルのロード
# ==========================================
model_save_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds_mlp = []
all_labels_mlp = []
success_vxs_mlp = []

print(f"MLP Baseline 最終統合評価を開始します...")

# CVで保存した全試合のMLPモデルをロードして評価
for test_match in match_ids:
    # データの抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # モデルの準備
    model = SimpleMLPClassifier(in_channels=7).to(device)
    model_path = os.path.join(model_save_dir, f'mlp_match_{test_match}.pth')

    if not os.path.exists(model_path):
        continue

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 物理的妥当性の計算
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 全ノードの平均vx（集団の推進力）
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # Successと予測した時のみ記録
                    success_vxs_mlp.append(avg_vx)

            all_preds_mlp.extend(pred.cpu().numpy())
            all_labels_mlp.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 2. レポート出力
# ==========================================
print("\n" + "="*60)
print(" MLP Baseline 最終クラシフィケーションレポート")
print("="*60)
print(classification_report(all_labels_mlp, all_preds_mlp, target_names=['Fail', 'Success'], zero_division=0))

print("\n" + "="*60)
print(" 物理的整合性 検証 (MLP vs 物理法則)")
print("="*60)
if len(success_vxs_mlp) > 0:
    avg_vx_mlp = np.mean(success_vxs_mlp)
    pos_ratio_mlp = np.sum(np.array(success_vxs_mlp) > 0) / len(success_vxs_mlp)

    print(f"MLPが『成功』と予測したシーンの平均 vx: {avg_vx_mlp:.4f} m/s")
    print(f"右向き(攻撃方向)への推進力割合: {pos_ratio_mlp*100:.1f} %")

    # 考察用コメント
    if pos_ratio_mlp > 0.80:
        print(">> 考察: MLPは極めて高い確率で『右への速度』を成功の根拠としています。")
        print(">> これは空間構造を無視し、単純な物理量のみに依存している可能性を示唆します。")
else:
    print("Success予測なし")

# 混同行列の表示
cm = confusion_matrix(all_labels_mlp, all_preds_mlp)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (MLP Baseline)')
plt.show()

PFI

In [ ]:
import copy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import f1_score

# ==========================================
# 1. 確定した特徴量名の定義
# ==========================================
feature_names = [
    'Position_X',        # Index 0
    'Position_Y',        # Index 1
    'Velocity_X',        # Index 2
    'Velocity_Y',        # Index 3
    'Distance_to_Goal',  # Index 4
    'Distance_to_Ball',  # Index 5
    'Team_Flag'          # Index 6
]

def calculate_pfi_refined(model, loader, device, feature_names, model_type="GNN"):
    model.eval()
    y_true, y_pred = [], []

    # --- Step 1: ベースラインのF1スコアを計算 ---
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    baseline_f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"[{model_type}] Baseline F1: {baseline_f1:.4f}")

    importance_scores = {}

    # --- Step 2: 各特徴量を順番にシャッフルして影響を測定 ---
    for i, f_name in enumerate(feature_names):
        shuffled_f1_list = []

        # 3回試行して平均をとる（安定化のため）
        for seed in range(3):
            y_true_s, y_pred_s = [], []
            with torch.no_grad():
                for data in loader:
                    # データをコピーして特定の特徴量だけシャッフル
                    data_s = copy.deepcopy(data).to(device)
                    # ノード単位でシャッフル（全選手のその項目だけをバラバラにする）
                    perm = torch.randperm(data_s.x.size(0))
                    data_s.x[:, i] = data_s.x[perm, i]

                    out_s = model(data_s)
                    y_true_s.extend(data_s.y.view(-1).cpu().numpy())
                    y_pred_s.extend(out_s.argmax(dim=1).cpu().numpy())

            shuffled_f1 = f1_score(y_true_s, y_pred_s, zero_division=0)
            shuffled_f1_list.append(shuffled_f1)

        # 重要度 = ベースラインF1 - シャッフル後F1（下がれば下がるほど重要）
        importance_scores[f_name] = baseline_f1 - np.mean(shuffled_f1_list)
        print(f"  > Done: {f_name}")

    return importance_scores

# ==========================================
# 2. 実行（MLPとPIGNNの両方で回す）
# ==========================================

# 1. PIGNN (Alpha=1.0) の計算
# ※ 既にメモリ上にある最新の PIGNN モデルと test_loader を使用
print("\n PIGNN (Alpha=1.0) の重要度を算出中...")
pfi_pignn = calculate_pfi_refined(model, test_loader, device, feature_names, "PIGNN")

# 2. MLP の計算
# ※ 先ほど作成した mlp_model と test_loader を使用
print("\n MLP Baseline の重要度を算出中...")
# --- MLPモデルの器を再作成 ---
mlp_model = SimpleMLPClassifier(in_channels=7).to(device)

# --- Match 1 など、特定の学習済み重みをロード ---
# (PFIはテストデータとモデルのペアが必要なため、保存したファイルを指定します)
mlp_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline/mlp_match_1.pth"

if os.path.exists(mlp_path):
    mlp_model.load_state_dict(torch.load(mlp_path, map_location=device))
    print(f" MLP Model loaded from: {mlp_path}")

    # --- 改めて PFI を実行 ---
    print("\n MLP Baseline の重要度を算出中...")
    pfi_mlp = calculate_pfi_refined(mlp_model, test_loader, device, feature_names, "MLP")
else:
    print(f" ファイルが見つかりません: {mlp_path}")

# ==========================================
# 3. 卒論用グラフの作成
# ==========================================
df_pfi = pd.DataFrame({
    'PIGNN (α=1.0)': pfi_pignn,
    'MLP (Baseline)': pfi_mlp
})

# 横棒グラフで比較
df_pfi.plot(kind='barh', figsize=(10, 6), width=0.8)
plt.axvline(0, color='black', linewidth=0.8)
plt.title('Permutation Feature Importance: PIGNN vs MLP', fontsize=14)
plt.xlabel('Drop in F1-score (Higher means more important)', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import copy

# ==========================================
# 設定：検証モード（カンニングをオフにする）
# ==========================================
CHEATING_OFF = True  # True にすると Index 4 (Distance_to_Goal) を 0 に固定
# PIGNN(α=1.0) または MLP を指定して比較してください
MODEL_TYPE = "PIGNN" # "PIGNN" または "MLP"
FIXED_ALPHA = 1.0     # PIGNN の場合はフォルダ特定に使用

if MODEL_TYPE == "PIGNN":
    alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}"
    model_load_dir = os.path.join("/content/drive/MyDrive/GNN_Football_Analysis/Models", alpha_folder)
else:
    model_load_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_preds, all_labels = [], []

print(f" [{MODEL_TYPE}] 評価開始 (Distance_to_Goal 遮断: {CHEATING_OFF})")

for test_match in match_ids:
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # モデルの初期化とロード
    if MODEL_TYPE == "PIGNN":
        model = PIGNNClassifier(hidden_channels=64).to(device)
        model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    else:
        model = SimpleMLPClassifier(in_channels=7).to(device)
        model_path = os.path.join(model_load_dir, f'mlp_match_{test_match}.pth')

    if not os.path.exists(model_path): continue
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)

            # --- 【重要】入力データの加工 ---
            if CHEATING_OFF:
                # データをコピーし、Index 4 (Distance_to_Goal) を 0.0 で上書き
                # これにより、モデルは「ゴールに近いかどうか」の情報を使えなくなる
                data.x[:, 4] = 0.0

            out = model(data)
            pred = out.argmax(dim=1)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 結果表示
# ==========================================
print("\n" + "="*60)
status = "CLEAN (Cheating Off)" if CHEATING_OFF else "RAW (Cheating On)"
print(f"  {MODEL_TYPE} 最終レポート - {status}")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=['Fail', 'Success'], zero_division=0))

# 論文(cite: 115)の指標に基づき、F1スコアの低下率を記録しておくと考察に便利です
final_f1 = f1_score(all_labels, all_preds, zero_division=0)
print(f"\nFinal F1 Score: {final_f1:.4f}")

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import copy

# ==========================================
# 設定：検証モード（カンニングをオフにする）
# ==========================================
CHEATING_OFF = True  # True にすると Index 4 (Distance_to_Goal) を 0 に固定
# PIGNN(α=1.0) または MLP を指定して比較してください
MODEL_TYPE = "MLP" # "PIGNN" または "MLP"
FIXED_ALPHA = 1.0     # PIGNN の場合はフォルダ特定に使用

if MODEL_TYPE == "PIGNN":
    alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}"
    model_load_dir = os.path.join("/content/drive/MyDrive/GNN_Football_Analysis/Models", alpha_folder)
else:
    model_load_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_preds, all_labels = [], []

print(f" [{MODEL_TYPE}] 評価開始 (Distance_to_Goal 遮断: {CHEATING_OFF})")

for test_match in match_ids:
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # モデルの初期化とロード
    if MODEL_TYPE == "PIGNN":
        model = PIGNNClassifier(hidden_channels=64).to(device)
        model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    else:
        model = SimpleMLPClassifier(in_channels=7).to(device)
        model_path = os.path.join(model_load_dir, f'mlp_match_{test_match}.pth')

    if not os.path.exists(model_path): continue
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)

            # --- 【重要】入力データの加工 ---
            if CHEATING_OFF:
                # データをコピーし、Index 4 (Distance_to_Goal) を 0.0 で上書き
                # これにより、モデルは「ゴールに近いかどうか」の情報を使えなくなる
                data.x[:, 4] = 0.0

            out = model(data)
            pred = out.argmax(dim=1)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 結果表示
# ==========================================
print("\n" + "="*60)
status = "CLEAN (Cheating Off)" if CHEATING_OFF else "RAW (Cheating On)"
print(f"  {MODEL_TYPE} 最終レポート - {status}")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=['Fail', 'Success'], zero_division=0))

# 論文(cite: 115)の指標に基づき、F1スコアの低下率を記録しておくと考察に便利です
final_f1 = f1_score(all_labels, all_preds, zero_division=0)
print(f"\nFinal F1 Score: {final_f1:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ==========================================
# 1. 実験データの入力 (得られた数値を代入)
# ==========================================
# カンニングあり (Raw) の時の数値を以前の結果から推定・入力してください
# ここでは比較のために概算値を設定しています
labels = ['PIGNN (α=1.0)', 'MLP Baseline']

# F1スコアの変化
f1_cheating_on = [0.3654, 0.4200]  # カンニングあり
f1_cheating_off = [0.1646, 0.1475] # カンニングなし (今回の結果)

# 再現率 (Recall) の変化
recall_cheating_on = [0.65, 0.64]   # 推定値（PIGNN）, MLP（レポート値）
recall_cheating_off = [0.89, 0.23]  # 今回の結果

x = np.arange(len(labels))
width = 0.35

# ==========================================
# 2. グラフ描画：F1スコアの低下比較
# ==========================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# --- 左図：F1スコアの変化 ---
ax1.bar(x - width/2, f1_cheating_on, width, label='With Goal Distance', color='skyblue', alpha=0.6)
ax1.bar(x + width/2, f1_cheating_off, width, label='Without Goal Distance', color='blue')
ax1.set_ylabel('F1 Score')
ax1.set_title('Robustness Comparison: F1 Score Drop')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.legend()
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# --- 右図：再現率(Recall)の維持能力 ---
ax2.bar(x - width/2, recall_cheating_on, width, label='With Goal Distance', color='salmon', alpha=0.6)
ax2.bar(x + width/2, recall_cheating_off, width, label='Without Goal Distance', color='red')
ax2.set_ylabel('Recall (Success detection)')
ax2.set_title('Robustness Comparison: Recall (Tactical Awareness)')
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.legend()
ax2.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# ==========================================
# 3. 統計データのサマリー出力
# ==========================================
print("--- 考察用サマリー ---")
for i in range(len(labels)):
    drop = (f1_cheating_on[i] - f1_cheating_off[i]) / f1_cheating_on[i] * 100
    print(f"{labels[i]}: F1スコア低下率 {drop:.1f}%")

アテンション係数の可視化

アテンション係数の可視化のためにはアテンションを戻り値として返すように修正が必要。そのためにモデルを再定義する。

In [ ]:
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    # --- 引数 return_attention を追加 ---
    def forward(self, x, edge_index, pos, vel, return_attention=False):
        h = self.lin(x)
        # チームID(index 6)をメッセージパッシングに渡す
        out = self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 6:7])

        # 可視化モードの時は、内部で計算された alpha を取得する
        if return_attention:
            # 内部変数を保持するために一時的な保存が必要ですが、
            # シンプルにするため、ここでは出力と共にエッジインデックスと直近のアテンションを返せるように設計します
            return out, (edge_index, self._last_att if hasattr(self, '_last_att') else None)
        return out

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        is_teammate = (team_i == team_j).float()
        team_bias = torch.where(is_teammate > 0.5, 0.5, -0.5)

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        # 【可視化用】計算されたアテンションを一時保存
        self._last_att = alpha
        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    # --- 引数 return_attention を追加 ---
    def forward(self, data, return_attention=False):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        if return_attention:
            # conv1 からアテンションを抽出
            x, (edge_idx_out, att_weights) = self.conv1(x, edge_index, pos, vel, return_attention=True)
            x = F.elu(x)
            x = self.conv2(x, edge_index, pos, vel)
        else:
            x = F.elu(self.conv1(x, edge_index, pos, vel))
            x = self.conv2(x, edge_index, pos, vel)

        x_pool = global_mean_pool(x, batch)
        logits = F.log_softmax(self.lin(x_pool), dim=1)

        if return_attention:
            return logits, (edge_idx_out, att_weights)
        return logits

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
from torch_geometric.loader import DataLoader

def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    """
    PIGNNのアテンション係数と物理状態（座標・速度）をピッチ上に可視化する
    """
    model.eval()

    # データを1つだけのバッチとして扱う
    data_item = data_item.to(device)

    # モデルから予測結果とアテンション係数を抽出
    # ※ forwardメソッドが return_attention=True で (out, (edge_index, att_weights)) を返す前提
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 1. 座標と速度の復元 ---
    # 正規化された値 (-1~1) を実際のピッチサイズ (105m x 68m) に戻す
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()

    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5  # X座標: -52.5 to 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0  # Y座標: -34.0 to 34.0

    # 速度ベクトルの描画用スケーリング（1秒間の移動距離を強調）
    vel_plot = vel * 3.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- 2. サッカー場の描画 (芝生の色 #2e7d32) ---
    ax.set_facecolor('#2e7d32')
    # 外枠
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))

    # センターライン & センターサークル
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))

    # ペナルティエリア
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- 3. アテンション係数（黄色い光 #FFFF00）の描画 ---
    # 全エッジのうち、影響力の強い上位のエッジを光の線で表現
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 95) # 上位○%のエッジのみ表示
        max_att = att_weights.max()
        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                # 強さに応じて透明度(alpha)を変化させる
                alpha_val = (att_weights[i] - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.7, lw=2.0 + alpha_val*3, zorder=2)

    # --- 4. 選手とボールの描画 ---
    team_ids = data_item.x[:, 6].cpu().numpy() # 7次元目のTeam_Flagを取得
    num_nodes = pos.shape[0]

    for i in range(num_nodes):
        # チームに応じた色分け
        if team_ids[i] == 2.0: # ボール (Gold)
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃チーム (Blue #0288d1)
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備チーム (Red #d32f2f)
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        # 本体描画
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=z)

        # 速度ベクトル矢印 (物理的推進力の可視化)
        if team_ids[i] != 2.0: # 選手のみ矢印を表示
            ax.arrow(pos_plot[i, 0], pos_plot[i, 1], vel_plot[i, 0], vel_plot[i, 1],
                     head_width=1.0, head_length=1.2, fc='white', ec='white', alpha=0.5, zorder=z-1)

    # 情報テキストの表示
    label_str = "SUCCESS" if label == 1 else "FAILURE"
    pred_str = "SUCCESS" if pred == 1 else "FAILURE"
    match_result = "CORRECT" if label == pred else "INCORRECT"

    ax.set_title(f"{title}\nActual: {label_str} | Predicted: {pred_str} ({prob:.1%})\nResult: {match_result}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60)
    ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 5. 実行：成功シーンと失敗シーンの自動抽出と可視化
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case = None
    failure_case = None

    model.eval()
    for data in data_list:
        with torch.no_grad():
            out, _ = model(data.to(device), return_attention=True)
            pred = out.argmax(dim=1).item()
            label = data.y.item()

            # AIが正解したケースから1つずつピックアップ
            if pred == label:
                if label == 1 and success_case is None:
                    success_case = data
                elif label == 0 and failure_case is None:
                    failure_case = data

        if success_case and failure_case:
            break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# 実行コマンド
# 1. モデルのインスタンス化
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

# 2. 重みのロード
# alpha_param は現在のモデルに定義されていないため、strict=False で無視させます
pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_1_0/pignn_testmatch_1.pth"
state_dict = torch.load(pignn_path, map_location=device)
pignn_model.load_state_dict(state_dict, strict=False)
print(" 重みのロード完了。可視化準備が整いました。")

# 3. 可視化実行
# 先ほど作成した visualize_pignn_attention_v3 や run_comparison_visualizer を実行
run_comparison_visualizer(pignn_model, all_data_list, device)

速度ベクトルも表示

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 可視化メイン関数 (速度ベクトル改善版)
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # 推論とアテンション抽出
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- サッカー場の描画 ---
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- アテンションの描画 (zorder=2) ---
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位5%を表示
        max_att = att_weights.max()
        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                alpha_val = (att_weights[i] - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.8, lw=2.0 + alpha_val*4, zorder=2)

    # --- 速度ベクトルと選手の描画 (zorder=10-20) ---
    team_ids = data_item.x[:, 6].cpu().numpy()
    num_nodes = pos.shape[0]

    #  速度描画用の設定
    vel_scale = 20.0 # 矢印をはっきり見せるためのスケーリング

    for i in range(num_nodes):
        if team_ids[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        #  改善：ax.quiver で速度を最前面に描画 (zorder=20)
        if team_ids[i] != 2.0:
            ax.quiver(pos_plot[i, 0], pos_plot[i, 1],
                      vel[i, 0], vel[i, 1],
                      color='white', alpha=0.9,
                      angles='xy', scale_units='xy', scale=1/vel_scale,
                      width=0.005, headwidth=4, headlength=5, zorder=20)

        # 選手ノード本体
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=15)

    # テキスト表示
    res_text = "SUCCESS" if pred == 1 else "FAILURE"
    match_status = "CORRECT" if label == pred else "INCORRECT"
    ax.set_title(f"{title}\nActual: {'SUCCESS' if label==1 else 'FAILURE'} | Predicted: {res_text} ({prob:.1%})\nResult: {match_status}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. 自動抽出 & 実行ループ
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case, failure_case = None, None
    model.eval()

    for data in data_list:
        with torch.no_grad():
            # データを転送
            d_gpu = data.to(device)
            out, _ = model(d_gpu, return_attention=True)
            pred = out.argmax(dim=1).item()
            label = d_gpu.y.item()

            # 正解シーンの中から抽出
            if pred == label:
                if label == 1 and success_case is None: success_case = data
                elif label == 0 and failure_case is None: failure_case = data

        if success_case and failure_case: break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# ==========================================
# 3. モデルロード & 実行
# ==========================================
# 1. インスタンス化
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

# 2. ロード (strict=False)
pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_1_0/pignn_testmatch_1.pth"
if os.path.exists(pignn_path):
    state_dict = torch.load(pignn_path, map_location=device)
    pignn_model.load_state_dict(state_dict, strict=False)
    print(" PIGNN重みのロードに成功しました。")
    # 3. 実行
    run_comparison_visualizer(pignn_model, all_data_list, device)
else:
    print(f" ファイルが見つかりません: {pignn_path}")

アテンションの数値の追加

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
import os
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 可視化 & 数値抽出メイン関数
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # --- 推論とアテンション抽出 ---
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標と速度の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- サッカー場の描画 (zorder=0-1) ---
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- 2. アテンション係数の描画 & 数値出力 (zorder=2) ---
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位2%に絞り込み
        max_att = att_weights.max()

        print(f"\n{'='*50}")
        print(f" {title} - TOP 2% ATTENTION DETAILS")
        print(f"{'='*50}")
        print(f"{'Source':<8} | {'Dest':<8} | {'Weight':<10} | {'Team Relation'}")
        print(f"{'-'*50}")

        # チームID取得 (0:攻撃, 1:守備, 2:ボール)
        team_ids = data_item.x[:, 6].cpu().numpy()

        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                weight = att_weights[i]

                # チーム関係の言語化
                rel = "Teammate" if team_ids[src] == team_ids[dst] else "Opponent"
                if team_ids[src] == 2.0: rel = "Ball -> Player"

                # コンソールに数値を表示
                print(f"Node {src:2d} -> Node {dst:2d} | {weight:.4f}     | {rel}")

                # 描画
                alpha_val = (weight - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.8, lw=2.0 + alpha_val*4, zorder=2)

    # --- 3. 選手と速度ベクトルの描画 (zorder=10-20) ---
    num_nodes = pos.shape[0]
    vel_scale = 15.0 # 速度ベクトルの視認性を確保

    for i in range(num_nodes):
        if team_ids[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃チーム
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備チーム
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        # 速度ベクトル (ax.quiver で確実に描画)
        if team_ids[i] != 2.0:
            ax.quiver(pos_plot[i, 0], pos_plot[i, 1],
                      vel[i, 0], vel[i, 1],
                      color='white', alpha=0.9,
                      angles='xy', scale_units='xy', scale=1/vel_scale,
                      width=0.005, headwidth=4, headlength=5, zorder=20)

        # 選手ノード
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=15)

    # --- タイトルと表示設定 ---
    res_text = "SUCCESS" if pred == 1 else "FAILURE"
    match_status = "CORRECT" if label == pred else "INCORRECT"
    ax.set_title(f"{title}\nActual: {'SUCCESS' if label==1 else 'FAILURE'} | Predicted: {res_text} ({prob:.1%})\nResult: {match_status}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. 自動比較実行ループ
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case, failure_case = None, None
    model.eval()

    for data in data_list:
        with torch.no_grad():
            d_gpu = data.to(device)
            out, _ = model(d_gpu, return_attention=True)
            pred = out.argmax(dim=1).item()
            label = d_gpu.y.item()

            if pred == label:
                if label == 1 and success_case is None: success_case = data
                elif label == 0 and failure_case is None: failure_case = data

        if success_case and failure_case: break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# ==========================================
# 3. 実行セクション
# ==========================================
# PIGNNモデルの準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_1_0/pignn_testmatch_1.pth"
if os.path.exists(pignn_path):
    state_dict = torch.load(pignn_path, map_location=device)
    pignn_model.load_state_dict(state_dict, strict=False)
    print(" PIGNN重みのロードに成功しました。")

    # 実行
    run_comparison_visualizer(pignn_model, all_data_list, device)
else:
    print(f" パスが見つかりません: {pignn_path}")